In [ ]:
import os
import zipfile
import urllib.request
import pickle
import numpy as np
import re
from tqdm import tqdm
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from keras.models import Model
from nltk.translate.bleu_score import corpus_bleu
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
BASE_DIR = 'content/flickr8k/'
IMAGES_DIR = os.path.join(BASE_DIR, 'Images/')
WORKING_DIR = 'content/'

os.makedirs(IMAGES_DIR, exist_ok=True)

images_url = 'https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip'
captions_url = 'https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip'

image_zip = os.path.join(BASE_DIR, 'Flickr8k_Dataset.zip')
caption_zip = os.path.join(BASE_DIR, 'Flickr8k_text.zip')

if not os.path.exists(IMAGES_DIR) or len(os.listdir(IMAGES_DIR)) == 0:
    print("📥 Downloading Flickr8k dataset (Images + Captions)...")
    urllib.request.urlretrieve(images_url, image_zip)
    urllib.request.urlretrieve(captions_url, caption_zip)

    print("📦 Extracting files...")
    with zipfile.ZipFile(image_zip, 'r') as zip_ref:
        zip_ref.extractall(BASE_DIR)
    with zipfile.ZipFile(caption_zip, 'r') as zip_ref:
        zip_ref.extractall(BASE_DIR)
    print("✅ Dataset ready!")

In [ ]:
model = VGG16(weights='imagenet')
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

In [ ]:
features = {}
directory = os.path.join(BASE_DIR, 'Images')

for img_name in tqdm(os.listdir(directory)):
    img_path = os.path.join(directory, img_name)
    image = load_img(img_path, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, *image.shape))
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)
    image_id = img_name.split('.')[0]
    features[image_id] = feature

# Save extracted features
pickle.dump(features, open(os.path.join(WORKING_DIR, 'features.pkl'), 'wb'))


In [ ]:
captions_path = os.path.join(BASE_DIR, 'Flickr8k.token.txt')
with open(captions_path, 'r') as f:
    captions_doc = f.read()

mapping = {}
for line in tqdm(captions_doc.split('\n')):
    tokens = line.split()
    if len(line) < 2:
        continue
    image_id, image_caption = tokens[0], tokens[1:]
    image_id = image_id.split('.')[0]
    caption = ' '.join(image_caption)
    if image_id not in mapping:
        mapping[image_id] = []
    mapping[image_id].append(caption)

def clean(mapping):
    for key, captions in mapping.items():
        for i in range(len(captions)):
            caption = captions[i].lower()
            caption = re.sub('[^a-z]', ' ', caption)
            caption = re.sub('\s+', ' ', caption)
            caption = 'startseq ' + caption.strip() + ' endseq'
            captions[i] = caption

clean(mapping)

In [ ]:
all_captions = []
for key in mapping:
    for caption in mapping[key]:
        all_captions.append(caption)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)
vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(caption.split()) for caption in all_captions)
print('📊 Vocab Size:', vocab_size)
print('🧾 Max Caption Length:', max_length)

In [ ]:
image_ids = list(mapping.keys())
split = int(len(image_ids) * 0.90)
train = image_ids[:split]
test = image_ids[split:]

In [ ]:
def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    X1, X2, y = list(), list(), list()
    n = 0
    while True:
        for key in data_keys:
            # Skip if feature not found for this image
            if key not in features:
                continue

            captions = mapping[key]
            for caption in captions:
                seq = tokenizer.texts_to_sequences([caption])[0]
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    X1.append(features[key][0])
                    X2.append(in_seq)
                    y.append(out_seq)

            if len(X1) == batch_size:
                yield ([np.array(X1), np.array(X2)], np.array(y))
                X1, X2, y = list(), list(), list()


In [ ]:
inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.4)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)

decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

print(model.summary())

In [ ]:
epochs = 1
batch_size = 32
steps = len(train) // batch_size

generator = data_generator(train, mapping, features, tokenizer, max_length, vocab_size, batch_size)
model.fit(generator, epochs=epochs, steps_per_epoch=steps, verbose=1)

In [ ]:
def idx_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def predict_caption(model, image, tokenizer, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([image, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = idx_to_word(yhat, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text

In [ ]:
sample_img = list(mapping.keys())[0] + '.jpg'
img_path = os.path.join(IMAGES_DIR, sample_img)
image = Image.open(img_path)
plt.imshow(image)
plt.axis('off')
plt.show()

image_feature = features[sample_img.split('.')[0]]
y_pred = predict_caption(model, image_feature, tokenizer, max_length)
print("🧾 Predicted Caption:\n", y_pred)

In [ ]:
# ============================================================
# SIMPLE IMAGE CAPTIONING MODEL (VGG16 + LSTM)
# ============================================================

import os
import zipfile
import numpy as np
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences, to_categorical
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from keras.models import Model

# ============================================================
# 1️⃣ SETUP PATHS
# ============================================================
BASE_DIR = "content/keras_image/"
WORKING_DIR = "content/"
os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, "Images"), exist_ok=True)

# ============================================================
# 2️⃣ DOWNLOAD DATASET (Flickr8k)
# ============================================================
!wget -q https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip -O content/Flickr8k_Dataset.zip
!wget -q https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip -O content/Flickr8k_text.zip

with zipfile.ZipFile("content/Flickr8k_Dataset.zip", "r") as zip_ref:
    zip_ref.extractall(os.path.join(BASE_DIR, "Images"))
with zipfile.ZipFile("content/Flickr8k_text.zip", "r") as zip_ref:
    zip_ref.extractall(BASE_DIR)

print("✅ Dataset ready!")

# ============================================================
# 3️⃣ EXTRACT FEATURES USING VGG16
# ============================================================
model = VGG16()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

features = {}
directory = os.path.join(BASE_DIR, "Images", "Flicker8k_Dataset")

for img_name in tqdm(os.listdir(directory)):
    path = os.path.join(directory, img_name)
    image = load_img(path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)
    features[img_name.split(".")[0]] = feature

pickle.dump(features, open(os.path.join(WORKING_DIR, "features.pkl"), "wb"))
print("✅ Features extracted and saved!")

# ============================================================
# 4️⃣ LOAD CAPTIONS AND CLEAN
# ============================================================
with open(os.path.join(BASE_DIR, "Flickr8k.token.txt"), "r") as f:
    captions_doc = f.read()

mapping = {}
for line in captions_doc.split("\n"):
    if len(line) < 2:
        continue
    image_id, caption = line.split("\t")
    image_id = image_id.split("#")[0]
    caption = "startseq " + caption.lower().replace("[^a-z]", "") + " endseq"
    mapping.setdefault(image_id, []).append(caption)

# ============================================================
# 5️⃣ TOKENIZE CAPTIONS
# ============================================================
all_captions = [c for caps in mapping.values() for c in caps]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)
vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(c.split()) for c in all_captions)

# ============================================================
# 6️⃣ TRAIN-TEST SPLIT
# ============================================================
image_ids = list(mapping.keys())
split = int(len(image_ids) * 0.9)
train, test = image_ids[:split], image_ids[split:]

# ============================================================
# 7️⃣ DATA GENERATOR (Simplified + Key Check)
# ============================================================
def data_generator(keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    while True:
        X1, X2, y = [], [], []
        for key in keys:
            if key not in features:
                continue
            for caption in mapping[key]:
                seq = tokenizer.texts_to_sequences([caption])[0]
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    X1.append(features[key][0])
                    X2.append(in_seq)
                    y.append(out_seq)
                    if len(X1) == batch_size:
                        yield ([np.array(X1), np.array(X2)], np.array(y))
                        X1, X2, y = [], [], []

# ============================================================
# 8️⃣ DEFINE MODEL
# ============================================================
inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.4)(inputs1)
fe2 = Dense(256, activation="relu")(fe1)

inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)

decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation="relu")(decoder1)
outputs = Dense(vocab_size, activation="softmax")(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss="categorical_crossentropy", optimizer="adam")
print(model.summary())

# ============================================================
# 9️⃣ TRAIN MODEL (Short Training)
# ============================================================
generator = data_generator(train, mapping, features, tokenizer, max_length, vocab_size, batch_size=32)
steps = len(train) // 32
model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)

# ============================================================
# 🔟 PREDICTION FUNCTION
# ============================================================
def idx_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def predict_caption(model, image, tokenizer, max_length):
    in_text = "startseq"
    for _ in range(max_length):
        seq = tokenizer.texts_to_sequences([in_text])[0]
        seq = pad_sequences([seq], maxlen=max_length)
        yhat = model.predict([image, seq], verbose=0)
        word = idx_to_word(np.argmax(yhat), tokenizer)
        if word is None:
            break
        in_text += " " + word
        if word == "endseq":
            break
    return in_text

# ============================================================
# 🖼️ TEST SAMPLE
# ============================================================
sample_img = test[0]
image = Image.open(os.path.join(directory, sample_img + ".jpg"))
plt.imshow(image)

caption = predict_caption(model, features[sample_img], tokenizer, max_length)
print("🧠 Predicted Caption:\n", caption)